In [1]:
import pandas as pd
ds = pd.read_csv('Panel.csv') 

def extract_details(row):
    try:
        # Parse JSON
        data = json.loads(row['response'])
        
        # Extract line items
        line_items = data.get('data', {}).get('line_items', [])
        
        extracted_data = []
        for item in line_items:
            # Extract relevant fields
            extracted_data.append({
                'claim_data_id': row['claim_data_id'],
                'intimation_number': row['intimation_number'],
                'bill_item_name': item.get('data', {}).get('description', {}).get('value', ''),
                'vitraya_master_category': item.get('data', {}).get('vitraya_master_category', {}).get('value', ''),
                'tariff_item_name': item.get('data', {}).get('tariff', {}).get('tariff_line_item', ''),
                'hospital_charged_amount': item.get('data', {}).get('amount', {}).get('value', 0.0),
                'quantity': item.get('data', {}).get('quantity', {}).get('value', 0.0),
                'irdai_payable': item.get('data', {}).get('tariff', {}).get('irdai_payable', False),
                'admissible_amount': item.get('data', {}).get('tariff', {}).get('admissible_amount', 0.0)
            })
        
        return extracted_data
    except json.JSONDecodeError:
        return []


ds_extracted = ds.apply(extract_details, axis=1)
ds_expanded = pd.DataFrame([item for sublist in ds_extracted for item in sublist])

ds_expanded_sorted = ds_expanded.sort_values(
    by=['claim_data_id', 'intimation_number', 'bill_item_name'],
    ascending=[True, True, True]  # True means ascending order
)

ds_expanded['Line Item Status'] = ds_expanded['bill_item_name'] == ds_expanded['tariff_item_name']

df = ds_expanded.copy()

 #Step 1: Calculate total hospital charged amount and total admissible amount for 'Medicines & Consumables'
med_consumables = df[df['vitraya_master_category'] == 'Medicines & Consumables']
med_consumables_totals = med_consumables.groupby('claim_data_id').agg(
    admissible_amount_total_of_medicine_items=pd.NamedAgg(column='admissible_amount', aggfunc='sum'),
    total_hospital_charged_amount_meds=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum')
).reset_index()

# Step 2: Calculate total hospital charged amount and total admissible amount for all items
all_items_totals = df.groupby('claim_data_id').agg(
    total_hospital_charged_amount_all=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum'),
    total_admissible_amount_all=pd.NamedAgg(column='admissible_amount', aggfunc='sum')
).reset_index()

# Step 3: Merge all totals into a single DataFrame
final_totals = pd.merge(med_consumables_totals, all_items_totals, on='claim_data_id', how='outer')

# Fill NaNs with 0 if necessary
final_totals.fillna(0, inplace=True)

In [8]:
ds_expanded

,claim_data_id,intimation_number,bill_item_name,vitraya_master_category,tariff_item_name,hospital_charged_amount,quantity,irdai_payable,admissible_amount
0,12736,1112485205093,Admission Fees,Service Tax & Other Taxes,Admission Fees,0.00,1.0,False,0.00
1,12736,1112485205093,I C U,Medicines & Consumables,u dic,0.00,1.0,True,0.00
2,12736,1112485205093,Deluxe A/C Room,Room Rent,Delux Room AC/ NON AC,6000.00,3.0,True,6000.00
3,12736,1112485205093,Special with A/C Room,Room Rent,General Room AC/NON AC,0.00,1.0,True,0.00
4,12736,1112485205093,Special Room Non A/C,Room Rent,General Room AC/NON AC,0.00,1.0,True,0.00
...,...,...,...,...,...,...,...,...,...
1489,10996,1112485189493,PHARMACY BILL (Department) : Bill :11582425/17173,Service Tax & Other Taxes,PHARMACY BILL (Department) : Bill :11582425/17173,43728.07,1.0,True,43728.07
1490,10996,1112485189493,PHARMACY BILL (Department) : Bill :11582425/17175,Service Tax & Other Taxes,PHARMACY BILL (Department) : Bill :11582425/17175,14425.74,1.0,True,14425.74
1491,10996,1112485189493,PHARMACY BILL (Department) : Bill :11582425/17266,Service Tax & Other Taxes,PHARMACY BILL (Department) : Bill :11582425/17266,145818.63,1.0,True,145818.63
1492,10996,1112485189493,PHARMACY SALES RETURN : PHARMACY SALES RETURN,Medicines & Consumables,PHARMACY SALES RETURN : PHARMACY SALES RETURN,4340.45,1.0,True,4340.45


In [13]:
ds_expanded_sorted = ds_expanded.sort_values(
    by=['claim_data_id', 'intimation_number', 'bill_item_name'],
    ascending=[True, True, True]  # True means ascending order
)